# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [ ]:
%help

####  Run this cell to set up and start your interactive session.


In [1]:
%idle_timeout 2880
%glue_version 4.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.5 
Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 4.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 5
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 5
Idle Timeout: 2880
Session ID: 680564ab-d3eb-480d-86d8-7f982c6be2d3
Applying the following default arguments:
--glue_kernel_version 1.0.5
--enable-glue-datacatalog true
Waiting for session 680564ab-d3eb-480d-86d8-7f982c6be2d3 to get into ready status...
Session 680564ab-d3eb-480d-86d8-7f982c6be2d3 ha

#### Example: Create a DynamicFrame from a table in the AWS Glue Data Catalog and display its schema


In [2]:
data_path = "s3://sai-capstone/housing/raw/housing.csv"  
housing_df = spark.read.csv(data_path, header=True, inferSchema=True)

In [3]:
housing_df.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

In [4]:
housing_df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)


In [5]:
housing_df.describe().show()

+-------+-------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+---------------+
|summary|          longitude|         latitude|housing_median_age|       total_rooms|    total_bedrooms|        population|       households|     median_income|median_house_value|ocean_proximity|
+-------+-------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+---------------+
|  count|              20640|            20640|             20640|             20640|             20433|             20640|            20640|             20640|             20640|          20640|
|   mean|-119.56970445736148| 35.6318614341087|28.639486434108527|2635.7630813953488| 537.8705525375618|1425.4767441860465|499.5396802325581|3.8706710029070246|206855.81690891474|           null|
| stddev|  2.0035317

In [6]:
median_total_bedrooms = housing_df.approxQuantile("total_bedrooms", [0.5], 0.001)[0]
housing_df = housing_df.fillna({"total_bedrooms": median_total_bedrooms})

In [7]:
from pyspark.sql.functions import col, udf, mean, stddev
from pyspark.sql.types import DoubleType, IntegerType
@udf(DoubleType())
def rooms_per_household(rooms, households):
    return rooms / households

@udf(DoubleType())
def population_per_household(population, households):
    return population / households

@udf(DoubleType())
def bedrooms_per_room(bedrooms, rooms):
    return bedrooms / rooms

In [8]:
housing_df = housing_df.withColumn("rooms_per_household", rooms_per_household(col("total_rooms"), col("households")))
housing_df = housing_df.withColumn("population_per_household", population_per_household(col("population"), col("households")))
housing_df = housing_df.withColumn("bedrooms_per_room", bedrooms_per_room(col("total_bedrooms"), col("total_rooms")))



In [9]:
# Drop unnecessary column
housing_df = housing_df.drop("income_cat")

In [10]:
numeric_features = ["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population", "households", "median_income"]
for feature in numeric_features:
    mean_val = housing_df.agg(mean(col(feature))).collect()[0][0]
    stddev_val = housing_df.agg(stddev(col(feature))).collect()[0][0]
    housing_df = housing_df.withColumn(feature, (col(feature) - mean_val) / stddev_val)

In [11]:
output_path = "s3://sai-capstone/housing/prepared/houusing_prepared.csv"
housing_df.write.csv(output_path,mode='overwrite', header=True)

print("Data processing complete. Processed data saved to S3.")

Data processing complete. Processed data saved to S3.
